# OpenAI Assistant Functions
#### The purpose of this notebook is to store the code required to build assistants, threads, and the basis for communicating with them

In [1]:
# import statement and loading environment variables
import os
import pathlib
from google.cloud import storage 
import json
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()

### Create new Assistant and corresponding Thread

In [3]:
# Create new assistant and thread
import os
import pathlib
from google.cloud import storage 
import json
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()


def create_assistant(instructions, name, model, tools):
    my_assistant = openai.beta.assistants.create(
        instructions=instructions,
        name=name,
        model=model,
        tools=tools
    )

    thread = client.beta.threads.create()

    # Extracting the IDs and converting them to strings
    assistant_id = str(my_assistant.id)
    thread_id = str(thread.id)

    return assistant_id, thread_id

name = "Wilke"
model = "gpt-4"
tools = [{"type": "code_interpreter"}]
instructions = """
Create a function that takes a list of numbers and returns the sum of the list.
"""

assistant_id, thread_id = create_assistant(instructions, name, model, tools)

print("Assistant ID:", assistant_id)
print("Thread ID:", thread_id)


Assistant ID: asst_ftO8z80b5GznHsyYNxtnZYuf
Thread ID: thread_9KDuNzSuhvBpt5PCLbONasf2


### Create Standalone Thread

In [3]:
new_thread = client.beta.threads.create()
print("New Thread ID:", new_thread.id)

New Thread ID: thread_0xv4erIG7utTkd4dFtgHZRZN


### Send Message to the assistant

In [6]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
import time


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()



def send_message_to_assistant(thread_id, assistant_id, message, wait_time=5):
    # Send the message to the specified thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message
    )

    # Create a run for the assistant to process the message
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Wait for a short period to allow the assistant to respond
    time.sleep(wait_time)

    # Retrieve the messages after the wait time
    messages = client.beta.threads.messages.list(thread_id)

    # Find the latest assistant's response in the thread
    for msg in reversed(messages.data):
        if msg.role == 'assistant':
            return msg.content[0].text.value

    return "No response from the assistant."

# Example usage
message = "Hello, Assistant! What is the square root of 1533332244?"

response = send_message_to_assistant(thread_id, assistant_id, message)
print("Assistant's response:", response)


Assistant's response: The square root of 15333344 is approximately 3915.78.


### List all the messages from a thread

In [8]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()


def print_all_messages_in_thread(thread_id):
    # Retrieve the list of messages in the thread
    response_messages = client.beta.threads.messages.list(thread_id)

    # Iterate through the messages and print them
    for msg in response_messages:
        print("Message ID:", msg.id)
        print("Role:", msg.role)
        print("Content:", msg.content)
        print("Created At:", msg.created_at)
        print("-------------------------")

# # Example usage
thread_id = "thread_9KDuNzSuhvBpt5PCLbONasf2"
print_all_messages_in_thread(thread_id)


Message ID: msg_2Lqop8sp2KRqEeBSBhtO952E
Role: assistant
Content: [MessageContentText(text=Text(annotations=[], value="Yes, I'm here and ready to assist you. What can I help you with?"), type='text')]
Created At: 1702493321
-------------------------
Message ID: msg_l1HwXHtMTsckgOC34n8pWv92
Role: user
Content: [MessageContentText(text=Text(annotations=[], value='Working yet?'), type='text')]
Created At: 1702493320
-------------------------
Message ID: msg_xyjxHKUremCWSYeO1rQdiBF4
Role: assistant
Content: [MessageContentText(text=Text(annotations=[], value='Hello! How can I assist you today?'), type='text')]
Created At: 1702493063
-------------------------
Message ID: msg_jznVROvYuDAleTtHMXTrMzlj
Role: user
Content: [MessageContentText(text=Text(annotations=[], value='Hey'), type='text')]
Created At: 1702493061
-------------------------
Message ID: msg_xJsm8s7iu5M2s8q3GTHUzA7L
Role: assistant
Content: [MessageContentText(text=Text(annotations=[], value='I\'m here to help answer your ques